### Info

Author: Jelle Poland \
Citing: https://doi.org/10.3390/en16145264 \
License: ... \
Github: ...


### Initialisation


In [ ]:
# making things autorelad - needed for Jupyter Kernel
%load_ext autoreload
%autoreload 2
%matplotlib widget

# # define the right-path
import sys
import os
# TODO: remove this hardcoding
folder_path = '/home/jellepoland/surfdrive/phd/code/kitesim/'
os.chdir(folder_path)  # This should not be needed
sys.path.append(os.getcwd())



# Time to import the modules
from src.initialisation import load_surfplan, pulley_connectivity
from src.initialisation.input_classes import input_VSM , input_bridle_aero, input_structural_solver
from src.initialisation import input_particleSystem,particles_with_rotational_resistance
from src.particleSystem.ParticleSystem import ParticleSystem
from src.coupling import coupling_aero2struc, coupling_struc2aero
from src.structural import structural_model, structural_mesher
from src.solver import solver_main, solver_utils
from src.post_processing import functions_print, functions_plot, post_processing_utils, post_processing_main
from src.initialisation import actuation_relations
from src.aerodynamic import VSM, breukels_2D, plate_aero, bridle_line_system_aero, tether_aero

# from test import test_main
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import yaml
import importlib
import pytest
import pandas as pd
import dill

# {} gives units. {{}} is normal {} in Latexs
from IPython.display import display, Latex

## All Immutatbles are stored in the dataclass config, i.e. simulation settings and configuration of the kite 
from src.initialisation.yaml_loader import config

## Mutable variables
# Initializing Mutable Variables
points = config.kite.points_ini
# defining vel_app (vector of vel_app_norm)
vel_app = config.vel_wind - config.vel_kite

# TODO: remove this, has been taken into the run_simulation part
# if (
#     config.kite_name == "V3_25"
# ):  # TODO: Should be generated/imported from the surfplan file instead
#     # ACTUATION
#     depower_tape_extension = actuation_relations.up_to_ld(
#         config.u_p, config.depower_tape_extension_percentage
#     )  # [mm] depower-tape extension
#     # bridle_rest_lengths[1] corresponds to the depower tape
#     bridle_rest_lengths[1] += depower_tape_extension

# Should be the same for each kite
(
    connectivity_matrix,
    wing_connectivity,
) = input_particleSystem.define_connectivity_matrix(config)
params_dict = input_particleSystem.define_params(
    config, wing_connectivity, connectivity_matrix
)
initial_conditions = input_particleSystem.define_initial_conditions_kite(
    config)
points_between_dict = particles_with_rotational_resistance.extract_points_between_dict(
    config
)
if config.is_with_initial_plot:
    functions_plot.plot_initial_geometry(config, points_between_dict)

is_with_rotational_resistance = False
if config.kite_name == "V9_60C":
    is_with_rotational_resistance = True

if is_with_rotational_resistance:
    (
        leadingedge_rotational_resistance_dict,
        strut_rotational_resistance_dict,
    ) = particles_with_rotational_resistance.extract_rotational_resistances_dicts(
        points_between_dict, config
    )
    # first do the struts
    k_bend_strut = 1e10
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_strut,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        strut_rotational_resistance_dict,
    )
    # secondly do the leading-edge
    k_bend_leadingedge = 1e4
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_leadingedge,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        leadingedge_rotational_resistance_dict,
    )
# Should be the same for each kite
psystem = ParticleSystem(connectivity_matrix, initial_conditions, params_dict)

# Printing initial dimensions
print(f"scaling-factor: {config.geometric_scaling_factor}")
print(f'ref_chord: {config.kite.ref_chord:.2f}m')
print(f'wing_span: {config.kite.span:.2f}m')
print(f'wing height: {config.kite.height:.2f}m')
print(f'wing area: {config.kite.area_surface:.2f}m2')
print(f'projected_area: {config.kite.area_projected:.2f}m')
print(f'At 25m/s --> Reynolds number: {1e-6*(1.225*25*config.kite.ref_chord)/(1.8e-5):.2f}e6')

# Static-aero


In [ ]:
# external force
# Struc --> aero
points_left_to_right = coupling_struc2aero.order_struc_nodes_right_to_left(
    points, config.kite.connectivity.plate_point_indices
)
# Wing Aerodynamic
(
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    F_rel,
    ringvec,
    controlpoints,
    wingpanels,
    rings,
    coord_L,
    coord_refined,
) = VSM.calculate_force_aero_wing_VSM(points_left_to_right, vel_app, input_VSM)
# Aero --> struc
force_aero_wing = coupling_aero2struc.aero2struc(
    points,
    config.kite.connectivity.wing_ci,
    config.kite.connectivity.wing_cj,
    config.kite.connectivity.plate_point_indices,
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    ringvec,
    controlpoints,
)  # Put in the new positions of the points

# Bridle Aerodynamics
if config.is_with_aero_bridle:
    force_aero_bridle = bridle_line_system_aero.calculate_force_aero_bridle_thedens2022(
        points, vel_app, input_bridle_aero
    )
else:
    force_aero_bridle = [0]
force_aero = force_aero_wing + force_aero_bridle

functions_print.print_aero(points, vel_app, force_aero_wing, force_aero_bridle, config)

#######
points_wing = np.copy(points)
evaluation_point = np.array([max(points_wing[:, 0]) / 2, 0, max(points_wing[:, 2])])
evaluation_point = np.zeros(3)
for i, point in enumerate(points_wing):
    points_wing[i] = point - evaluation_point
    if i not in config.kite.connectivity.plate_point_indices:
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Full Wing on mid-span, mid-chord point --> Mx: {moments_wing[:,0].sum():.3f} My: {moments_wing[:,1].sum():.3f} Mz: {moments_wing[:,2].sum():.2f}"
)
for i, point in enumerate(points_wing):
    if points_wing[i][1] > 0:  # taking only the right wing
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_half_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Half Right Wing on mid-span, mid-chord point --> Mx: {moments_half_wing[:,0].sum():.3f} My: {moments_half_wing[:,1].sum():.3f} Mz: {moments_half_wing[:,2].sum():.2f}"
)
print(f"norm of Va: {np.linalg.norm(vel_app):.2f}m/s")
#######

# plotting
post_processing_main.plot(
    [
        [wingpanels, controlpoints, rings, coord_L, F_rel],
        config.kite.wing_rest_lengths_initial,
        config.kite.bridle_rest_lengths_initial,
    ],
    points,
    vel_app,
    config,
)

In [ ]:
wingpanels